In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset
import random
import os

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
# For reproducibility, you can define the following function to fix the random seeds.
def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


seed_everything(42)

In [ ]:
# Read the data
yelp_review = pd.read_csv('./data/train.csv', header=0)

# make the stars 0, 1, 2
yelp_review['stars'] = yelp_review['stars'] - 1

# test_data is for testing ONLY
test_data = pd.read_csv('./data/new_test.csv', header=0)
test_label = list(test_data['stars'])

# test data for kaggle submission
final_test_data = pd.read_csv('./data/test.csv', header=0)
final_test_data['ID'] = 0

In [ ]:
import torch
from transformers import AutoTokenizer, DebertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-large")


# Tokenize data
def preprocess(examples):
    tokenized = tokenizer(examples['text'], truncation=True, padding=True)
    return tokenized


yelp_review = yelp_review.rename(columns={'stars': 'label'})
test_data = test_data.rename(columns={'stars': 'label'})
final_test_data = final_test_data.rename(columns={'ID': 'label'})

dataset = Dataset.from_pandas(yelp_review)
test_dataset = Dataset.from_pandas(test_data)
fina_test_dataset = Dataset.from_pandas(final_test_data)

eval_dataset = test_dataset.map(preprocess, batched=True, remove_columns=["text"])
train_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])
fina_result_dataset = fina_test_dataset.map(preprocess, batched=True, remove_columns=["text"])

In [ ]:
num_labels = 3
class_names = [0, 1, 2]

# id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-large", id2label=id2label)

model.to(device)

In [ ]:
training_args = TrainingArguments(
    evaluation_strategy='steps',
    save_strategy="no",
    learning_rate=1e-6,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    output_dir='./result_debert'
)


def get_trainer(model):
    return Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator

    )

In [ ]:
# create the trainer
trainer = get_trainer(model)

iteration = 0

In [ ]:
while 1:
    trainer.train()
    iteration += 1
    print(iteration)

    # Calculate the accuracy on test set
    pred_output = trainer.predict(eval_dataset)
    test_label = list(pred_output[1])
    predictions = np.argmax(pred_output[0], axis=1)
    match = 0
    for i in range(len(predictions)):
        if predictions[i] == test_label[i]:
            match += 1
    current_acc = match / len(predictions)
    print(current_acc)

    # Output the prediction for kaggle submission
    final_output = trainer.predict(fina_result_dataset)

    final_predictions = np.argmax(final_output[0], axis=1)

    labels = range(1, len(final_predictions) + 1)
    predicted = []

    # add 1 back
    for j in final_predictions:
        predicted.append(j + 1)

    d = {'ID': list(labels), 'stars': predicted}
    df = pd.DataFrame(data=d)
    df_name = 'result_new' + str(iteration) + str(current_acc) + '.csv'
    df.to_csv(df_name, index=False)
